# 4.7: Deriving New Variables

In this exercise, I will derive new variables using:
-if-statements
-user-defined functions
-the `loc()` method
-for-loops based on the `ords_prods_merge` dataframe.

In [2]:
#Import libraries
import pandas as pd
import numpy as np
import os

In [3]:
#02. Setting main project path
#Base path to the Instacart project folder
path = r'/Users/jessduong/Documents/CF/Achievement 4_Python/12-2025 Instacart Basket Analysis/02 Data/Prepared Data/'

In [4]:
# Importing the ords_prods_merge dataframe from the Prepared Data folder

ords_prods_merge = pd.read_pickle(os.path.join(path, 'ords_prods_merge.pkl'))
# pd.read_pickle() loads a dataframe saved in pickle format (.pkl)
# os.path.join() safely constructs the full file path
# 'ords_prods_merge.pkl' must match the exact filename in your Prepared Data folder

In [5]:
# Creating a subset of the ords_prods_merge dataframe
# This subset includes only the first 1,000,000 rows for performance reasons

df = ords_prods_merge[:1000000]   # slice from the start up to (but not including) row 1,000,000

In [6]:
# Checking the first few rows to confirm the dataframe loaded correctly
ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_previous_order,first_order_flag,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days_2,busiest_period_of_day
0,2539329,1,prior,1,2,8,0.0,True,196,1,0,Soda,77.0,7.0,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2539329,1,prior,1,2,8,0.0,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,Mid-range product,Regularly busy,Regularly busy,Average orders
2,2539329,1,prior,1,2,8,0.0,True,12427,3,0,Original Beef Jerky,23.0,19.0,4.4,Low-range product,Regularly busy,Regularly busy,Average orders
3,2539329,1,prior,1,2,8,0.0,True,26088,4,0,Aged White Cheddar Popcorn,23.0,19.0,4.7,Low-range product,Regularly busy,Regularly busy,Average orders
4,2539329,1,prior,1,2,8,0.0,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,Low-range product,Regularly busy,Regularly busy,Average orders


In [7]:
# Checking the shape to confirm the number of rows and columns
df.shape

(1000000, 19)

In [8]:
# Define a function to classify products into price ranges

def price_label(row): #this function checks the price in each row and returns a label

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'     # Catch missing values or unexpected value

In [9]:
# Apply the function to create a new column called price_range

df['price_range'] = df.apply(price_label, axis=1)
#axis=1 means function is applied row-by-row

/var/folders/85/2tfx5b95721gp7lbbkvhcplw0000gn/T/ipykernel_76733/822000892.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [10]:
# Checking the distribution of the new price_range labels
df['price_range'].value_counts(dropna = False)

# value_counts() shows how many products fall into each category

price_range
Mid-range product    672525
Low-range product    314109
High range            12256
Not enough data        1110
Name: count, dtype: int64

In [11]:
# Checking that the total of all price_range labels equals the number of rows in df
df['price_range'].value_counts(dropna=False).sum()

np.int64(1000000)

In [12]:
# Checking the maximum price in the subset to explain the presence of "High range" products
df['prices'].max()

25.0

In [13]:
# Checking rows with unusually high prices to investigate potential placeholder/error values
df.loc[df['prices'] >= 1000, ['product_id', 'product_name', 'prices']].head(20)
# This filters for extreme prices and displays key identifying columns for inspection

,product_id,product_name,prices


In [14]:
# Replacing unrealistic price values with NaN

df.loc[df['prices'] > 100, 'prices'] = np.nan
# Prices above $100 are treated as data errors or placeholders

In [15]:
# Recreating the price_range column after cleaning price outliers

df['price_range'] = df.apply(price_label, axis=1)
# This re-applies the function using the updated prices column

/var/folders/85/2tfx5b95721gp7lbbkvhcplw0000gn/T/ipykernel_76733/939617723.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [16]:
# Verifying the updated distribution of price ranges
df['price_range'].value_counts(dropna=False)

price_range
Mid-range product    672525
Low-range product    314109
High range            12256
Not enough data        1110
Name: count, dtype: int64

After identifying unrealistic price values (e.g., prices above $100), these values were treated as missing data to prevent distortion of the price range variable. After reapplying the price range logic, the number of high-range products decreased slightly, while the number of “Not enough data” entries increased, confirming that only extreme outliers were affected and that valid high-priced products remained classified appropriately.

In [17]:
# Inspecting a few high-range items to confirm the values look reasonable
df.loc[df['prices'] > 15, ['product_id', 'product_name', 'prices']].head(10)

,product_id,product_name,prices
398,38293,Ground Turkey Breast,19.6
401,38293,Ground Turkey Breast,19.6
436,13198,85% Lean Ground Beef,23.2
465,13198,85% Lean Ground Beef,23.2
480,13198,85% Lean Ground Beef,23.2
503,13198,85% Lean Ground Beef,23.2
530,13198,85% Lean Ground Beef,23.2
542,13198,85% Lean Ground Beef,23.2
576,13198,85% Lean Ground Beef,23.2
610,13198,85% Lean Ground Beef,23.2


# Creating price range labels using `loc()`

Next, I used the `loc()` method to create the same type of price range labels. This approach is faster than using a user-defined function with `apply()` and avoids scanning the entire dataframe row-by-row.

In [18]:
# Creating price_range_loc labels for prices above 15
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [19]:
# Creating price_range_loc labels for prices between 5 and 15
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [20]:
# Creating price_range_loc labels for prices 5 or below
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [21]:
# Checking the distribution of the loc-based labels
df['price_range_loc'].value_counts(dropna = False)
# NaN values represent rows where prices are missing (NaN) and therefore cannot be classified

price_range_loc
Mid-range product     672525
Low-range product     314109
High-range product     12256
NaN                     1110
Name: count, dtype: int64

# Applying `loc()` to the full dataframe

After testing the logic on the one-million-row subset, I repeated the `loc()` process on the full `ords_prods_merge` dataframe.

In [22]:
# Cleaning unrealistic price values in the full dataframe (prices above $100 treated as missing)

ords_prods_merge.loc[ords_prods_merge['prices'] > 100, 'prices'] = np.nan
# This prevents extreme placeholder values from distorting the price range classification

In [23]:
# Verifying the maximum price after cleaning outliers in the full dataframe
# If cleaning worked, the max should now be <= 100 (or NaN-safe max)

ords_prods_merge['prices'].max()

25.0

In [24]:
# Creating price_range_loc labels on the full dataframe for prices above 15

ords_prods_merge.loc[ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [25]:
# Creating price_range_loc labels on the full dataframe for prices between 5 and 15

ords_prods_merge.loc[(ords_prods_merge['prices'] <= 15) & (ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [26]:
# Creating price_range_loc labels on the full dataframe for prices 5 or below

ords_prods_merge.loc[ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [27]:
# Checking the distribution of price_range_loc labels on the full dataframe

ords_prods_merge['price_range_loc'].value_counts(dropna=False)

price_range_loc
Mid-range product     21860860
Low-range product     10126321
High-range product      412551
NaN                      35327
Name: count, dtype: int64

# Creating summary variables using for-loops

In this section, I use a `for` loop combined with conditional logic to derive a new categorical variable based on the `orders_day_of_week` column.  
The goal is to classify each order as occurring on the busiest day, least busy day, or a regularly busy day.  

This approach allows the same logic to be applied repeatedly across all rows in the dataframe and provides an efficient way to create a summary variable that highlights weekly ordering patterns for further analysis.

In [28]:
# Checking the frequency of orders by day of week
# This helps identify the most and least busy days

ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0    6209808
1    5665951
6    4500391
2    4217868
5    4209603
3    3844175
4    3787263
Name: count, dtype: int64

In [29]:
# Create an empty list to store the classification for each row
# This list will end up having the same number of elements as rows in ords_prods_merge

result = []

# Loop through each value in the orders_day_of_week column
# Each value represents the day of the week an order was placed
for value in ords_prods_merge["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")    # If the value is 0 (Saturday), label it as the busiest day
  elif value == 4:
    result.append("Least busy")      # If the value is 4 (Wednesday), label it as the least busy day
  else:
    result.append("Regularly busy")  
    # All other days are labeled as regularly busy

In [30]:
# Printing the result list to verify the output of the for-loop
# This list contains one label per row in the dataframe
# Repeated values are expected because many rows share the same day of week
result

['Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy

In [31]:
# Confirming that the result list length matches the number of rows in the dataframe
len(result) == ords_prods_merge.shape[0]

True

In [32]:
# Checking the number of rows in the full ords_prods_merge dataframe
ords_prods_merge.shape

(32435059, 19)

In [33]:
# Adding the busiest_day labels to the dataframe
ords_prods_merge['busiest_day'] = result

In [34]:
# Checking the distribution of the busiest_day variable
ords_prods_merge['busiest_day'].value_counts(dropna = False)

busiest_day
Regularly busy    22437988
Busiest day        6209808
Least busy         3787263
Name: count, dtype: int64

In [35]:
# Checking the distribution of the original orders_day_of_week variable
ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0    6209808
1    5665951
6    4500391
2    4217868
5    4209603
3    3844175
4    3787263
Name: count, dtype: int64

# Validation Check

The frequency distribution of the `busiest_day` variable aligns with the distribution of `orders_day_of_week`:

- Orders labeled "Busiest day" correspond to rows where `orders_day_of_week == 0`
- Orders labeled "Least busy" correspond to rows where `orders_day_of_week == 4`
- All remaining days are categorized as "Regularly busy"

This confirms that the for-loop logic correctly mapped each row and that no rows were dropped or duplicated during the transformation.

In [36]:
# Final integrity check: ensuring total counts match
ords_prods_merge['busiest_day'].value_counts().sum() == ords_prods_merge.shape[0]

np.True_

# Step 2. Deriving New Variables (Instacart)

In this task, I extend the variable derivations from the Exercise by:
1. Updating the "busiest day" logic to capture the two busiest and two slowest days of the week.
2. Creating a new variable that summarizes order activity into three **time-of-day periods**: “Most orders,” “Average orders,” and “Fewest orders.”
3. Validating the new columns and exporting the updated dataframe to the Prepared Data folder as a pickle file.

In [37]:
# Checking the distribution of orders by day of week (0–6)
# This allows us to identify the 2 busiest days and the 2 slowest days
dow_counts = ords_prods_merge['orders_day_of_week'].value_counts()

In [38]:
# Identifying the two busiest day codes (highest order counts)
top2_days = dow_counts.head(2).index.tolist()

In [39]:
# Identifying the two slowest day codes (lowest order counts)
bottom2_days = dow_counts.tail(2).index.tolist()

In [40]:
# Printing the results for transparency / documentation
top2_days, bottom2_days

([0, 1], [3, 4])

In [41]:
# Creating a new column for the updated business request:
# - "Busiest days" for the 2 busiest day codes
ords_prods_merge.loc[ords_prods_merge['orders_day_of_week'].isin(top2_days), 'busiest_days_2'] = 'Busiest days'

In [42]:
# Creating a new column for the updated business request:
# - "Slowest days" for the 2 slowest day codes
ords_prods_merge.loc[ords_prods_merge['orders_day_of_week'].isin(bottom2_days), 'busiest_days_2'] = 'Slowest days'

In [43]:
# Creating a new column for the updated business request:
# - "Regularly busy" for all other day codes
ords_prods_merge.loc[~ords_prods_merge['orders_day_of_week'].isin(top2_days + bottom2_days), 'busiest_days_2'] = 'Regularly busy'

In [44]:
# Checking the frequency of the new busiest_days_2 column
# This confirms that the column contains the expected 3 categories
ords_prods_merge['busiest_days_2'].value_counts(dropna=False)

busiest_days_2
Regularly busy    12927862
Busiest days      11875759
Slowest days       7631438
Name: count, dtype: int64

In [45]:
# Validating that the labels match the intended day codes
# This checks: every "Busiest days" row has a day code in top2_days, etc.

busiest_ok = ords_prods_merge.loc[ords_prods_merge['busiest_days_2'] == 'Busiest days', 'orders_day_of_week'].isin(top2_days).all()
slowest_ok = ords_prods_merge.loc[ords_prods_merge['busiest_days_2'] == 'Slowest days', 'orders_day_of_week'].isin(bottom2_days).all()

busiest_ok, slowest_ok

(np.True_, np.True_)

# Step 3. Observations: Two Busiest and Two Slowest Days

Using the frequency distribution of `orders_day_of_week`, the two busiest days were identified as day codes 0 and 1, while the two slowest days were identified as day codes 3 and 4. Based on these results, a new column called `busiest_days_2` was created with three categories: Busiest days, Slowest days, and Regularly busy.

The value counts confirm that all orders were successfully classified into one of these three groups. Additional validation checks show that all rows labeled as “Busiest days” and “Slowest days” correctly correspond to their intended day codes, confirming the accuracy of the classification logic.

# Step 4a. Create "busiest_period_of_day" (Most / Average / Fewest)

The senior technical officer wants a simplified view of demand by hour.
Instead of using raw hours, I categorize hours into:
- Most orders
- Average orders
- Fewest orders

Method:
1. Count orders per `order_hour_of_day`
2. Define "Most" as hours in the top quartile of hourly order volume
3. Define "Fewest" as hours in the bottom quartile
4. All remaining hours are labeled "Average"

In [46]:
# Counting how many orders occur in each hour (0–23)
hour_counts = ords_prods_merge['order_hour_of_day'].value_counts().sort_index()

In [47]:
# Using quartiles of the hourly counts to define "Most" and "Fewest"
# Top 25% of hourly volumes = "Most orders"
# Bottom 25% of hourly volumes = "Fewest orders"
q75 = hour_counts.quantile(0.75)
q25 = hour_counts.quantile(0.25)

In [48]:
# Identifying which hours fall into each group
most_hours = hour_counts[hour_counts >= q75].index.tolist()
fewest_hours = hour_counts[hour_counts <= q25].index.tolist()

In [49]:
# Printing for documentation
q25, q75, most_hours[:5], fewest_hours[:5]

(np.float64(272834.75),
 np.float64(2558354.0),
 [10, 11, 12, 13, 14],
 [0, 1, 2, 3, 4])

# Step 4b. Creating the busiest_period_of_day variable

To support operational planning, orders are grouped into broader time-of-day demand categories rather than individual hours.  
Using hourly order volume, each hour is classified as having Most orders, Average orders, or Fewest orders based on quartile thresholds.  
This new variable provides a simplified view of demand intensity throughout the day.

In [50]:
# Creating a new column to label demand intensity by hour of day
ords_prods_merge.loc[ords_prods_merge['order_hour_of_day'].isin(most_hours), 'busiest_period_of_day'] = 'Most orders'

In [51]:
ords_prods_merge.loc[ords_prods_merge['order_hour_of_day'].isin(fewest_hours), 'busiest_period_of_day'] = 'Fewest orders'

In [52]:
ords_prods_merge.loc[~ords_prods_merge['order_hour_of_day'].isin(most_hours + fewest_hours), 'busiest_period_of_day'] = 'Average orders'

In [53]:
# Checking by printing the frequency distribution of busiest_period_of_day
ords_prods_merge['busiest_period_of_day'].value_counts(dropna=False)

busiest_period_of_day
Most orders       16143548
Average orders    15694669
Fewest orders       596842
Name: count, dtype: int64

In [54]:
# Checking the original distribution of orders by hour for comparison
ords_prods_merge['order_hour_of_day'].value_counts().sort_index()

order_hour_of_day
0      218951
1      115787
2       69435
3       51321
4       53284
5       88064
6      290796
7      891951
8     1719991
9     2456751
10    2764476
11    2738647
12    2620898
13    2663346
14    2691598
15    2664583
16    2537506
17    2089510
18    1637956
19    1259416
20     977049
21     796379
22     634743
23     402621
Name: count, dtype: int64

# Cross-Check: Busiest Period of Day vs. Hourly Order Distribution

The frequency distribution of the `busiest_period_of_day` variable aligns with the underlying hourly order patterns.

From the original `order_hour_of_day` distribution, order volumes peak between late morning and early afternoon (approximately hours 10–14), which corresponds to the Most orders category. Early morning and late-night hours (roughly 0–4 and 22–23) show the lowest order volumes and are correctly grouped under Fewest orders.

All remaining hours fall into the Average orders category, confirming that the quartile-based classification accurately summarizes demand intensity across the day while reducing hourly granularity.

In [55]:
# Column creation check
ords_prods_merge.columns

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_previous_order', 'first_order_flag',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', 'price_range_loc', 'busiest_day',
       'busiest_days_2', 'busiest_period_of_day'],
      dtype='object')

In [56]:
'busiest_period_of_day' in ords_prods_merge.columns

True

# Exercise 4.7 – Summary of New Variables

In this exercise, new variables were derived from existing columns in the `ords_prods_merge` dataset to support clearer analysis and operational insights.
First, a `price_range` variable was created to classify products into low-, mid-, and high-price categories based on their listed prices. Unrealistic price values were identified and treated as missing to prevent distortion, and the classification logic was reapplied to ensure accurate labeling.
Next, customer demand patterns by day of the week were explored. The original `busiest_day` logic was extended to reflect updated stakeholder requirements by creating a new column, `busiest_days_2`, which identifies the two busiest days and the two slowest days of the week. The results were validated by cross-checking against the original distribution of orders by day.
Finally, hourly order volume was analyzed to create a simplified time-of-day demand indicator. Using quartiles of hourly order counts, a new variable called `busiest_period_of_day` was created with the labels Most orders, Average orders, and Fewest orders. The distribution of this variable was cross-checked against the raw hourly order distribution to confirm that peak and low-demand periods were correctly classified.
These derived variables provide clearer, higher-level insights into pricing, temporal demand patterns, and customer ordering behavior, supporting both analytical exploration and business decision-making.

In [57]:
# Exporting the updated dataframe with newly derived variables
ords_prods_merge.to_pickle(os.path.join(path, 'ords_prods_merge.pkl'))

In [59]:
ords_prods_merge.shape

(32435059, 19)